In [67]:
import os
from rclpy.serialization import deserialize_message, serialize_message
from rosbag2_py import SequentialReader, SequentialWriter, StorageOptions, ConverterOptions
from visualization_msgs.msg import Marker
from builtin_interfaces.msg import Duration
import numpy as np

In [101]:
# Helper function to update the marker message
def update_marker_message(marker_msg, new_id):
    marker_msg.id = new_id
    # marker_msg.lifetime = Duration(sec=0, nanosec=0)  # Infinite lifetime
    marker_msg.action = Marker.ADD
    marker_msg.ns = "fov_markers"  # Consistent namespace
    marker_msg.frame_locked = False
    return marker_msg

# Helper function to create axis markers
def create_axis_marker(parent_frame, child_frame, transform, axis_color, axis_vector, marker_id, stamp):
    marker = Marker()
    marker.header.frame_id = parent_frame
    marker.header.stamp.sec = int(stamp // 1e9)
    marker.header.stamp.nanosec = int(stamp % 1e9)
    marker.ns = f"tf_axis_{child_frame}"
    marker.id = marker_id
    marker.type = Marker.LINE_LIST
    marker.action = Marker.ADD
    marker.scale.x = 0.05
    marker.color.r = axis_color[0]
    marker.color.g = axis_color[1]
    marker.color.b = axis_color[2]
    marker.color.a = 1.0

    # Origin of the axis
    origin = Point(
        x=transform.transform.translation.x,
        y=transform.transform.translation.y,
        z=transform.transform.translation.z
    )

    # Extract the quaternion from the transform
    qx = transform.transform.rotation.x
    qy = transform.transform.rotation.y
    qz = transform.transform.rotation.z
    qw = transform.transform.rotation.w

    # Convert quaternion to rotation matrix
    rotation_matrix = np.array([
        [1 - 2 * (qy**2 + qz**2), 2 * (qx*qy - qz*qw), 2 * (qx*qz + qy*qw)],
        [2 * (qx*qy + qz*qw), 1 - 2 * (qx**2 + qz**2), 2 * (qy*qz - qx*qw)],
        [2 * (qx*qz - qy*qw), 2 * (qy*qz + qx*qw), 1 - 2 * (qx**2 + qy**2)]
    ])

    # Rotate the axis vector using the rotation matrix
    axis_vector_rotated = np.dot(rotation_matrix, np.array(axis_vector))

    # Compute the endpoint of the axis
    endpoint = Point(
        x=origin.x + axis_vector_rotated[0],
        y=origin.y + axis_vector_rotated[1],
        z=origin.z + axis_vector_rotated[2]
    )

    # Define the axis line
    marker.points.append(origin)
    marker.points.append(endpoint)

    return marker

In [99]:
# Paths
input_folder = "/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/dynus/for_visualization"
input_bag_path = f"{input_folder}/num_3"

# Output bag path
output_folder = os.path.join(input_folder, "output")
output_bag_path = f"{output_folder}/num_3_updated"

id_offset = 0

# Open reader and writer
reader = SequentialReader()
reader.open(StorageOptions(uri=input_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))

writer = SequentialWriter()
writer.open(StorageOptions(uri=output_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))


# Register all topics for writing
for topic_metadata in reader.get_all_topics_and_types():
    writer.create_topic(topic_metadata)

# Process messages
last_timestamp = None
while reader.has_next():
    topic, data, timestamp = reader.read_next()

    if topic == "/NX01/fov":
        marker_msg = deserialize_message(data, Marker)

        # Check if we should keep this message (1-second interval)
        if last_timestamp is None or (timestamp - last_timestamp) >= 0.5*1e9:  # 0.5 second in nanoseconds
            marker_msg = update_marker_message(marker_msg, id_offset)
            serialized_message = serialize_message(marker_msg)
            writer.write(topic, serialized_message, timestamp)

            last_timestamp = timestamp
            id_offset += 1

    else:
        # Write all other topics as they are
        writer.write(topic, data, timestamp)

[INFO] [1737235831.033550843] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/dynus/for_visualization/num_3/num_3_0.db3' for READ_ONLY.


RuntimeError: Database directory already exists (/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/dynus/for_visualization/output/num_3_updated), can't overwrite existing database

In [95]:
from rclpy.serialization import deserialize_message, serialize_message
from visualization_msgs.msg import Marker
from tf2_msgs.msg import TFMessage
from geometry_msgs.msg import Point
from rosbag2_py import SequentialReader, SequentialWriter, StorageOptions, ConverterOptions, TopicMetadata
import bisect

# Paths
input_folder = "/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/dynus/for_visualization/output"
input_bag_path = f"{input_folder}/num_3_updated/num_3_updated_0.db3"
output_folder = os.path.join(input_folder)
output_bag_path = f"{output_folder}/num_3_final"

# Open reader and writer
reader = SequentialReader()
reader.open(StorageOptions(uri=input_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))

writer = SequentialWriter()
writer.open(StorageOptions(uri=output_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))

# Register all original topics
for topic_metadata in reader.get_all_topics_and_types():
    writer.create_topic(topic_metadata)

# Register the new topic '/tf_axis'
tf_axis_metadata = TopicMetadata(
    name="/tf_axis",
    type="visualization_msgs/msg/Marker",
    serialization_format="cdr"
)
writer.create_topic(tf_axis_metadata)

# IDs of markers to keep
fov_ids_to_keep = [40, 50, 55, 60, 64, 69, 74, 81]
# Colors for X, Y, Z axes
axis_colors = {"x": (1.0, 0.0, 0.0), "y": (0.0, 1.0, 0.0), "z": (0.0, 0.0, 1.0)}
axis_vectors = {"x": (1.0, 0.0, 0.0), "y": (0.0, 1.0, 0.0), "z": (0.0, 0.0, 1.0)}
axis_id = 1000

# Process TF messages at 0.5-second intervals
last_processed_timestamp = None
interval_ns = int(0.5 * 1e9)  # 0.5 seconds in nanoseconds

# First pass: Collect FOV timestamps and TF messages
while reader.has_next():
    topic, data, timestamp = reader.read_next()

    if topic == "/NX01/fov":
        marker_msg = deserialize_message(data, Marker)

        if marker_msg.id in fov_ids_to_keep:
            writer.write(topic, serialize_message(marker_msg), timestamp)

    elif topic == "/tf":
        tf_msg = deserialize_message(data, TFMessage)

        # Check if this message is at least 0.5 seconds after the last processed message
        if last_processed_timestamp is None or (timestamp - last_processed_timestamp) >= interval_ns:
            last_processed_timestamp = timestamp

            # Create axis markers for each transform
            for transform in tf_msg.transforms:
                parent_frame = transform.header.frame_id
                child_frame = transform.child_frame_id

                # X, Y, Z axes
                for axis, vector in axis_vectors.items():
                    color = axis_colors[axis]
                    axis_marker = create_axis_marker(
                        parent_frame, child_frame, transform, color, vector, axis_id, timestamp
                    )
                    writer.write("/tf_axis", serialize_message(axis_marker), timestamp)
                    axis_id += 1
        
        # Write the original TF message
        writer.write(topic, data, timestamp)
    
    else:
        writer.write(topic, data, timestamp)

print(f"Processed bag saved at: {output_bag_path}")


[INFO] [1737235092.868912366] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/dynus/for_visualization/output/num_3_updated/num_3_updated_0.db3' for READ_ONLY.
[INFO] [1737235092.925590058] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/dynus/for_visualization/output/num_3_final/num_3_final_0.db3' for READ_WRITE.


Processed bag saved at: /media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/dynus/for_visualization/output/num_3_final


In [105]:
# For FASTER

# Paths
input_folder = "/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/faster/for_visualization"
input_bag_path = f"{input_folder}/num_8"

# Output bag path
output_folder = os.path.join(input_folder, "output")
output_bag_path = f"{output_folder}/num_8_updated"

id_offset = 0

# Open reader and writer
reader = SequentialReader()
reader.open(StorageOptions(uri=input_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))

writer = SequentialWriter()
writer.open(StorageOptions(uri=output_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))


# Register all topics for writing
for topic_metadata in reader.get_all_topics_and_types():
    writer.create_topic(topic_metadata)

# Process messages
last_timestamp = None
while reader.has_next():
    topic, data, timestamp = reader.read_next()

    if topic == "/NX01/fov":
        marker_msg = deserialize_message(data, Marker)

        # Check if we should keep this message (1-second interval)
        if last_timestamp is None or (timestamp - last_timestamp) >= 0.5*1e9:  # 0.5 second in nanoseconds
            marker_msg = update_marker_message(marker_msg, id_offset)
            serialized_message = serialize_message(marker_msg)
            writer.write(topic, serialized_message, timestamp)

            last_timestamp = timestamp
            id_offset += 1

    else:
        # Write all other topics as they are
        writer.write(topic, data, timestamp)

[INFO] [1737236803.158956306] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/faster/for_visualization/num_8/num_8_0.db3' for READ_ONLY.
[INFO] [1737236804.032974057] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/faster/for_visualization/output/num_8_updated/num_8_updated_0.db3' for READ_WRITE.


In [106]:
from rclpy.serialization import deserialize_message, serialize_message
from visualization_msgs.msg import Marker
from tf2_msgs.msg import TFMessage
from geometry_msgs.msg import Point
from rosbag2_py import SequentialReader, SequentialWriter, StorageOptions, ConverterOptions, TopicMetadata
import bisect

# Paths
input_folder = "/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/faster/for_visualization/output"
input_bag_path = f"{input_folder}/num_8_updated/num_8_updated_0.db3"
output_folder = os.path.join(input_folder)
output_bag_path = f"{output_folder}/num_8_final"

# Open reader and writer
reader = SequentialReader()
reader.open(StorageOptions(uri=input_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))

writer = SequentialWriter()
writer.open(StorageOptions(uri=output_bag_path, storage_id="sqlite3"),
            ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr"))

# Register all original topics
for topic_metadata in reader.get_all_topics_and_types():
    writer.create_topic(topic_metadata)

# Register the new topic '/tf_axis'
tf_axis_metadata = TopicMetadata(
    name="/tf_axis",
    type="visualization_msgs/msg/Marker",
    serialization_format="cdr"
)
writer.create_topic(tf_axis_metadata)

# IDs of markers to keep
fov_ids_to_keep = [37, 50, 56, 61, 67, 74, 78]
# Colors for X, Y, Z axes
axis_colors = {"x": (1.0, 0.0, 0.0), "y": (0.0, 1.0, 0.0), "z": (0.0, 0.0, 1.0)}
axis_vectors = {"x": (1.0, 0.0, 0.0), "y": (0.0, 1.0, 0.0), "z": (0.0, 0.0, 1.0)}
axis_id = 1000

# Process TF messages at 0.5-second intervals
last_processed_timestamp = None
interval_ns = int(0.5 * 1e9)  # 0.5 seconds in nanoseconds

# First pass: Collect FOV timestamps and TF messages
while reader.has_next():
    topic, data, timestamp = reader.read_next()

    if topic == "/NX01/fov":
        marker_msg = deserialize_message(data, Marker)

        if marker_msg.id in fov_ids_to_keep:
            writer.write(topic, serialize_message(marker_msg), timestamp)

    elif topic == "/tf":
        tf_msg = deserialize_message(data, TFMessage)

        # Check if this message is at least 0.5 seconds after the last processed message
        if last_processed_timestamp is None or (timestamp - last_processed_timestamp) >= interval_ns:
            last_processed_timestamp = timestamp

            # Create axis markers for each transform
            for transform in tf_msg.transforms:
                parent_frame = transform.header.frame_id
                child_frame = transform.child_frame_id

                # X, Y, Z axes
                for axis, vector in axis_vectors.items():
                    color = axis_colors[axis]
                    axis_marker = create_axis_marker(
                        parent_frame, child_frame, transform, color, vector, axis_id, timestamp
                    )
                    writer.write("/tf_axis", serialize_message(axis_marker), timestamp)
                    axis_id += 1
        
        # Write the original TF message
        writer.write(topic, data, timestamp)
    
    else:
        writer.write(topic, data, timestamp)

print(f"Processed bag saved at: {output_bag_path}")


[INFO] [1737236927.752745425] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/faster/for_visualization/output/num_8_updated/num_8_updated_0.db3' for READ_ONLY.
[INFO] [1737236927.809677630] [rosbag2_storage]: Opened database '/media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/faster/for_visualization/output/num_8_final/num_8_final_0.db3' for READ_WRITE.


Processed bag saved at: /media/kkondo/T7/dynus/tro_paper/yaw_optimization/benchmark2/bags/faster/for_visualization/output/num_8_final
